# Exploring propagation of record changes across databases and annotations

In [1]:
# For each record in SwissProt
#    Extract Taxa
#    Extract ProteinName and Subname
#    Extract 

In [2]:
# Imports
from Bio import SeqIO
import gzip
import progressbar

In [3]:
# Index UniProt proteins so that we can access via their accessions. 
#  - Only needs to be run once. Takes a few minutes. 

data_dir = '../data/'
up_gz = data_dir+'uniprot_sprot.dat.bgz'
up_db = data_dir+'uniprot_sprot.dat.biopy_db'
uniprot = SeqIO.index_db(index_filename=up_db, filenames=up_gz, format="swiss")


In [3]:
# Index gb_files
gb_file=data_dir+"gbbct60.seq.gz"
#h = gzip.open(gb_file, "rt")
#r =SeqIO.parse(h, "gb").__next__()

In [ ]:
# Create a mapping from GenBank and RefSeq to SwissProt record. 
from datetime import datetime
from netdbqual import utils; classify_acc(acc)
print(datetime.now().strftime("%H:%M:%S"))
gb_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:4]=="EMBL"}
print(datetime.now().strftime("%H:%M:%S"))
rs_sp_map={id.split(":")[1]:r  for r in uniprot for id in uniprot[r].dbxrefs if id[0:6]=="RefSeq"}

def get_dbxrefs(r):
    return [id for id in r.dbxrefs if id[0:6]=="RefSeq" or id[0:4]=="EMBL"]

print(datetime.now().strftime("%H:%M:%S"))
dbxref_tuples=[{db_acc.split(":")[0]:(db_acc.split(":")[1], uniprot[acc])}  for acc in uniprot for db_acc in get_dbxrefs(uniprot[acc]) ]

print(datetime.now().strftime("%H:%M:%S"))
gb_sp_map={accs['EMBL'] for accs in dbxref_tuples if 'EMBL' in accs}
rs_sp_map={accs['RefSeq'] for accs in dbxref_tuples if 'RefSeq' in accs}
print(datetime.now().strftime("%H:%M:%S"))


In [4]:
import re 
exec(open("../netdbqual/utils.py").read())

gb_file="gbbct15.seq.gz"
gbs_iter=SeqIO.parse(gzip.open(data_dir + gb_file, "rt"), "gb")
#gbs_cut=[gbs_iter.__next__() for i in range(300)]

#For each genome sequence,
gb_file_no_ext=gb_file.split(".")[0]


In [7]:
#exec(open("../netdbqual/utils.py").read())
#exec(open("../netdbqual/parser.py").read())
import glob
import re 

from os import path
import sys
sys.path.append('../')
import netdbqual.utils as u
import netdbqual.parser as parser
gb_file="gbbct*.seq.gz"
from importlib import reload  # Python 3.4+
reload(parser)


#for i,f in enumerate(glob.glob(data_dir + gb_file)):
#f="../../220411_ExamineFunction/data/EC3_4_11_4/EC_3_4_11_4_SP_sequence.gpk"
f=data_dir + "tmp_EC_3_4_11_4_partial_sequence.gpk"
if 1:
    #gbs_iter=SeqIO.parse(gzip.open(f, "rt"), "gb")
    gbs_iter=SeqIO.parse(f, "gb")
    gb_file_no_ext=path.basename(f).split(".")[0]
    #print(i)
    
    nodes=[]
    edges=[]
    with open(data_dir + gb_file_no_ext+'_node.csv', "w") as node_file:
        with open(data_dir + gb_file_no_ext+'_edge.csv', "w") as edge_file:            
            for j,r in enumerate(gbs_iter):
                # look at all CDS 
                print(j)
                (db,seq_type)=u.classify_acc(r.id)
                if seq_type=="nucleotide":
                    nodes,edges=parser.parse_gb_nuc(r)
                elif db=="uniprot" and seq_type=="protein":                    
                    nodes,edges=parser.parse_gb_protein_gff(r)
                elif seq_type=="protein":                    
                    nodes,edges=parser.parse_gb_protein_gff(r)                
                else:
                    print("Problem")
                    sdfsdfsd

                if edges:
                    edge_file.write('\n'.join(edges)+"\n")

                node_file.write('\n'.join(nodes)+"\n")
                

0


In [ ]:
# For each set of genbank records (gb*.seq.gz)
#  Extract record
#  Download history
#  